In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf RoBERTa-NL2SQL

GIT_PATH = "https://github.com/DebadityaPal/RoBERTa-NL2SQL"
!git clone "{GIT_PATH}"
%cd RoBERTa-NL2SQL

Cloning into 'RoBERTa-NL2SQL'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 32 (delta 9), reused 7 (delta 3), pack-reused 0
Unpacking objects: 100% (32/32), done.
/content/RoBERTa-NL2SQL


Downloading the dependencies

In [ ]:
!pip install SQLAlchemy==1.3.20
!pip install records==0.5.3
!pip install transformers==3.4.0

Let us import the libraries that we will be using

In [ ]:
import load_data
import torch
import json,argparse
import load_model
import roberta_training
import corenlp_local
import seq2sql_model_testing
import seq2sql_model_training_functions
import model_save_and_infer
import dev_function
import infer_functions
import time
import os
import nltk

from dbengine_sqlnet import DBEngine
from torchsummary import summary
from tqdm.notebook import tqdm
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


The following cell will set the PyTorch device to a GPU which enables us to use it during runtime.

In [ ]:
device = torch.device("cuda")
device

device(type='cuda')

## Loading Data From Files

In [ ]:
path_wikisql = "/content/drive/My Drive/RoBERTa NL2SQL"
BATCH_SIZE = 16

train_data, train_table, dev_data, dev_table, train_loader, dev_loader = load_data.get_data(path_wikisql, batch_size = BATCH_SIZE)
test_data,test_table,test_loader = load_data.get_test_data(path_wikisql, batch_size = BATCH_SIZE)
zero_data,zero_table,zero_loader = load_data.get_zero_data(path_wikisql, batch_size = BATCH_SIZE)    # Data to test Zero Shot Learning

## Loading Models

In [ ]:
roberta_model, tokenizer, configuration = load_model.get_roberta_model()          # Loads the RoBERTa Model
seq2sql_model = load_model.get_seq2sql_model(configuration.hidden_size)           # Loads the LSTM based submodels

## Loading the Pre trained weights, skip the below cell if you want to train the model from scratch

In [ ]:
path_roberta_pretrained = path_wikisql + "/model_roberta_best.pt"
path_model_pretrained = path_wikisql + "/model_best.pt"

if torch.cuda.is_available():
    res = torch.load(path_roberta_pretrained)
else:
    res = torch.load(path_roberta_pretrained, map_location='cpu')

roberta_model.load_state_dict(res['model_roberta'])

if torch.cuda.is_available():
    res = torch.load(path_model_pretrained)
else:
    res = torch.load(path_model_pretrained, map_location='cpu')

seq2sql_model.load_state_dict(res['model'])

<All keys matched successfully>

## Loading the Model Optimizers

##### RoBERTa: Adam Optimizer with learning rate = 0.00001
##### SubModels: Adam Optimizer with learning rate = 0.001

In [ ]:
model_optimizer, roberta_optimizer = load_model.get_optimizers(seq2sql_model , roberta_model)

## Below we define a function that prints the metrics in a readable format

In [ ]:
def print_result(epoch, acc, dname):
    ave_loss, acc_sc, acc_sa, acc_wn, acc_wc, acc_wo, acc_wvi, acc_wv, acc_lx, acc_x = acc

    print(f'{dname} results ------------')
    print(
        f" Epoch: {epoch}, ave loss: {ave_loss}, acc_sc: {acc_sc:.3f}, acc_sa: {acc_sa:.3f}, acc_wn: {acc_wn:.3f}, \
        acc_wc: {acc_wc:.3f}, acc_wo: {acc_wo:.3f}, acc_wvi: {acc_wvi:.3f}, acc_wv: {acc_wv:.3f}, acc_lx: {acc_lx:.3f}, acc_x: {acc_x:.3f}"
    )

## Training the Model

In [ ]:
EPOCHS = 5

In [ ]:
# acc_lx_t_best = 0.693             # Creats checkpoint so that a worse model does not get saved
acc_lx_t_best = 0  
epoch_best = 0                   
for epoch in range(EPOCHS):
    acc_train = dev_function.train( seq2sql_model, roberta_model, model_optimizer, roberta_optimizer, tokenizer, configuration, path_wikisql, train_loader)
    acc_dev, results_dev, cnt_list = dev_function.test(seq2sql_model, roberta_model, model_optimizer, tokenizer, configuration, path_wikisql, dev_loader, mode="dev")
    print_result(epoch, acc_train, 'train')
    print_result(epoch, acc_dev, 'dev')
    acc_lx_t = acc_dev[-2]
    if acc_lx_t > acc_lx_t_best:                  # IMPORTANT : Comment out this whole if block if you are using a shortcut to the original
        acc_lx_t_best = acc_lx_t                  #             Drive Folder, otherwise an error will stop the execution of the code.
        epoch_best = epoch                        #             You cannot edit the files in the original folder
                                                  #             Download and Upload a separate copy to change the files.
          
        # save best model
        state = {'model': seq2sql_model.state_dict()}
        torch.save(state, os.path.join(path_wikisql, 'model_best.pt'))

        state = {'model_roberta': roberta_model.state_dict()}
        torch.save(state, os.path.join(path_wikisql, 'model_roberta_best.pt'))

    print(f" Best Dev lx acc: {acc_lx_t_best} at epoch: {epoch_best}")

  0%|          | 0/3523 [00:00<?, ?it/s]

  0%|          | 0/527 [00:00<?, ?it/s]

train results ------------
 Epoch: 0, ave loss: 0.5897787607783868, acc_sc: 0.912, acc_sa: 0.882, acc_wn: 0.953,         acc_wc: 0.804, acc_wo: 0.886, acc_wvi: 0.808, acc_wv: 0.810, acc_lx: 0.563, acc_x: 0.632
dev results ------------
 Epoch: 0, ave loss: None, acc_sc: 0.965, acc_sa: 0.899, acc_wn: 0.977,         acc_wc: 0.899, acc_wo: 0.956, acc_wvi: 0.000, acc_wv: 0.911, acc_lx: 0.748, acc_x: 0.804
 Best Dev lx acc: 0.7475359220995131 at epoch: 0


  0%|          | 0/3523 [00:00<?, ?it/s]

  0%|          | 0/527 [00:00<?, ?it/s]

...

  0%|          | 0/527 [00:00<?, ?it/s]

train results ------------
 Epoch: 3, ave loss: 0.143965756310253, acc_sc: 0.959, acc_sa: 0.906, acc_wn: 0.981,         acc_wc: 0.915, acc_wo: 0.969, acc_wvi: 0.953, acc_wv: 0.953, acc_lx: 0.777, acc_x: 0.831
dev results ------------
 Epoch: 3, ave loss: None, acc_sc: 0.968, acc_sa: 0.907, acc_wn: 0.985,         acc_wc: 0.927, acc_wo: 0.973, acc_wvi: 0.000, acc_wv: 0.940, acc_lx: 0.789, acc_x: 0.844
 Best Dev lx acc: 0.7942049637810237 at epoch: 2


  0%|          | 0/3523 [00:00<?, ?it/s]

  0%|          | 0/527 [00:00<?, ?it/s]

train results ------------
 Epoch: 4, ave loss: 0.12956564127909725, acc_sc: 0.961, acc_sa: 0.907, acc_wn: 0.984,         acc_wc: 0.921, acc_wo: 0.973, acc_wvi: 0.959, acc_wv: 0.959, acc_lx: 0.787, acc_x: 0.841
dev results ------------
 Epoch: 4, ave loss: None, acc_sc: 0.971, acc_sa: 0.906, acc_wn: 0.985,         acc_wc: 0.930, acc_wo: 0.975, acc_wvi: 0.000, acc_wv: 0.950, acc_lx: 0.798, acc_x: 0.849
 Best Dev lx acc: 0.7983612397577485 at epoch: 4


## Testing The Model

In [ ]:
acc_dev, results_dev, _ = dev_function.test(seq2sql_model, roberta_model, model_optimizer, tokenizer, configuration, path_wikisql, dev_loader, mode="dev")
acc_test, results_test, _ = dev_function.test(seq2sql_model, roberta_model, model_optimizer, tokenizer, configuration, path_wikisql, test_loader, mode="test")
acc_zero, results_zero, _ = dev_function.test(seq2sql_model, roberta_model, model_optimizer, tokenizer, configuration, path_wikisql, zero_loader, mode="test")

print_result('test', acc_dev, 'dev')
print_result('test', acc_test, 'test')
print_result('test', acc_zero, 'zero')

  0%|          | 0/527 [00:00<?, ?it/s]

  0%|          | 0/993 [00:00<?, ?it/s]

  0%|          | 0/450 [00:00<?, ?it/s]

dev results ------------
 Epoch: test, ave loss: None, acc_sc: 0.971, acc_sa: 0.906, acc_wn: 0.985,         acc_wc: 0.930, acc_wo: 0.975, acc_wvi: 0.000, acc_wv: 0.950, acc_lx: 0.798, acc_x: 0.849
test results ------------
 Epoch: test, ave loss: None, acc_sc: 0.966, acc_sa: 0.907, acc_wn: 0.979,         acc_wc: 0.922, acc_wo: 0.967, acc_wvi: 0.000, acc_wv: 0.944, acc_lx: 0.790, acc_x: 0.844
zero results ------------
 Epoch: test, ave loss: None, acc_sc: 0.966, acc_sa: 0.901, acc_wn: 0.978,         acc_wc: 0.917, acc_wo: 0.962, acc_wvi: 0.000, acc_wv: 0.935, acc_lx: 0.774, acc_x: 0.833


## Test You Own Queries!

In [ ]:
nlu = "Which year did the band release the Song 'Wake me Up'?"

# Specify the Table Schema
table_id = '1-10015132-16'
headers = ['Band', 'Song', 'Studio', 'Year', 'Awards']
types = ['text', 'text', 'text', 'text', 'text']

In [ ]:
pr_sql_i =  infer_functions.infer(
                nlu,
                table_id, headers, types, tokenizer, 
                seq2sql_model, roberta_model, configuration, max_seq_length=222,
                num_target_layers=2,
                beam_size=4
            )

START ============================================================= 
[['Band', 'Song', 'Studio', 'Year', 'Awards']]
nlu: ["Which year did the band release the Song 'Wake me Up'?"]
pr_sql_i : [{'agg': 0, 'sel': 3, 'conds': []}]
pr_sql_q : [['SELECT (Year) FROM 1-10015132-16']]
---------------------------------------------------------------------
